
#Work 3 Hybrid Summarization with SumBasic and Pegasus-large Out-of-the-Box (claims/description sections)

Prior to this worksheet the claims and description sections of organic chemistry patents from HUPD were condensed using extractive summarization (SumBasic technique). We now try to form better summaries with Pegasus-large.

## Setup

In [ ]:
#install libraries
!pip install -q datasets
!pip install -q transformers
!pip install --quiet --upgrade accelerate
!pip install -q sentencepiece
!pip install -q evaluate
!pip install -q rouge_score

In [ ]:
# Install Packages

#standard data science libraries
import pandas as pd
import numpy as np
import random
import matplotlib

#visualization
import matplotlib.pyplot as plt
from pprint import pprint
from IPython.display import display, HTML

#datasets
import datasets
from datasets import load_from_disk
from datasets import load_dataset, load_metric

#transformers
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

# PyTorch
import torch
from torch.utils.data import DataLoader

#rouge
import evaluate

In [ ]:
# This cell will authenticate you and mount your Drive in the Colab.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
save_dir="/content/drive/MyDrive/W266/HUPD"

Mounted at /content/drive


##Load Data

In [ ]:
extracted_train = load_from_disk(save_dir + '/extracted_train_sample_cd')

In [ ]:
extracted_train

Dataset({
    features: ['patent_number', 'decision', 'title', 'abstract', 'cpc_label', 'ipc_label', 'filing_date', 'patent_issue_date', 'date_published', 'examiner_id', 'claims_desc', 'extractive_summaries'],
    num_rows: 720
})

##Prepare Data for Summmarization


In [ ]:
#so that we look at the same ones every time.
random_indices = [515, 175, 380, 364, 552, 238, 558, 6, 692, 504]

## Summarizing Extractive Summaries with Pegasus-large


In [ ]:
#specify base model
model_name = "google/pegasus-large"

#specify compute device (GPU if available)
device = "cuda" if torch.cuda.is_available() else "cpu"

#specify tokenizer
tokenizer = PegasusTokenizer.from_pretrained(model_name)

#instantiate model
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/3.09k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.encoder.embed_positions.weight', 'model.decoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/260 [00:00<?, ?B/s]

In [ ]:
#create a function to tokenize the data
def tokenize_input(input,column_name):
    tokenized = tokenizer(input[column_name], truncation=True, padding='max_length', return_tensors="pt") #.to(device)
    tokenized['input_ids'] = tokenized['input_ids'].squeeze()
    tokenized['attention_mask'] = tokenized['attention_mask'].squeeze()
    return tokenized

### Summarize Extracted Text using Pegasus Large

In [ ]:
# Tokenize training set
tokenized_input = extracted_train.map(lambda obs: tokenize_input(obs,'extractive_summaries'))

In [ ]:
#set format to Torch
tokenized_input.set_format(type='torch', columns=['input_ids', 'attention_mask']) #device = device to load the torch formatted tensors to your GPU.

#set up DataLoader
batch_size = 5 #note that the standard batch size for the DataLoader is 200
train_dataloader = DataLoader(tokenized_input, batch_size = batch_size) #num_workers = 4 << increase if you need higher speed.


In [ ]:
summarized_texts = []
for batch in iter(train_dataloader):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    summary = model.generate(input_ids = input_ids,
                             attention_mask = attention_mask,
                             max_length = 150
                             )
    summarized_text = tokenizer.batch_decode(summary, skip_special_tokens=False)
    summarized_texts.extend(summarized_text)

In [ ]:
#Generate Rouge Scores
rouge = evaluate.load('rouge')
predictions = summarized_texts
references = tokenized_input['abstract']
results = rouge.compute(predictions=predictions,
                        references=references)
print(results)

{'rouge1': 0.22505036775063114, 'rouge2': 0.08565479109805345, 'rougeL': 0.15808531994130587, 'rougeLsum': 0.15792722163262335}


In [ ]:
# Randomly sample example paies
sampled_summarized_texts = [summarized_texts[i] for i in random_indices]
sampled_abstracts = [extracted_train['abstract'][i] for i in random_indices]

# Zip the lists together
comparisons = list(zip(sampled_summarized_texts, sampled_abstracts))

# Create a pandas DataFrame from the zipped lists
comparison_df = pd.DataFrame(comparisons, columns=['predicted_summary', 'abstract'])

# Display df
display(HTML(comparison_df.to_html()))

,predicted_summary,abstract
0,"<pad>The method of claim 19, wherein the naturally occurring fatty acids and esters comprise chicken fat, beef tallow, yellow grease, choice white grease, lard, soybean oil, corn oil, cottonseed oil, canola oil, sunflower oil, palm oil, rapeseed oil, or a combination of any two or more thereof; and the at least 75 wt % even carbon number paraffins selected from the C12-C24 range comprise n-hexadecane and n-octadecane. The hydrogen-rich gas 105 is supplied at a rate of about 3,000 to about 10,000 SCF/bbl (volume gas per volume biological feedstock). The diameter of bubble column reactor vessel 104 is selected such that the gas flow rate is in the churn-</s>","Paraffin compositions including mainly even carbon number paraffins, and a method for manufacturing the same, is disclosed herein. In one embodiment, the method involves contacting naturally occurring fatty acid/glycerides with hydrogen in a slurry bubble column reactor containing bimetallic catalysts with equivalent particle diameters from about 10 to about 400 micron. The even carbon number compositions are particularly useful as phase change material."
1,"<pad>Then, a mixture with a purity of about 70.0%, 71.0%, 73.0%, 74.0%, 75.0%, 76.0%, 77.0%, 78.0%, 79.0%, 80.0%, 81.0%, 82.0%, 83.0%, 84.0%, 85.0%, 86.0%, 87.0%, 88.5%, 89.0%, 89.5%, 90.0%, about 90.5%, about 91.0%, about 91.5%, about 92.0%, about 92.5%, about 93.0%, about 93.5%, about 94.0%, about 94.5%, about 95.0%, about 95.5%, about 96.0%, about 96.5%, about 97.0%, about 97.5%, about 98</s>","The present application relates to a method for photochlorination, and specifically to photochlorination by a photochemical reaction of an aromatic compound with gaseous chlorine so as to prepare a trichloromethyl-substituted benzene, and to a method using bis-(trichloromethyl)-benzene as the trichloromethyl-substituted benzene to prepare by further reaction bis-(chloroformyl)-benzene. Through the control of temperature, illuminance and consumption of gaseous chlorine, the method of this application can greatly improve the purity of trichloromethyl-substituted benzene and further prepare polymer-grade bis-(chloroformyl)-benzene with low cost. The present application also relates to a method for purifying trichloromethyl-substituted benzene, and specifically to a method for purifying trichloromethyl-substituted benzene via molecular distillation. The present application further relates to a photochlorination reactor for use in photochlorination reactions (such as those of the present application)."
2,"<pad>Human CD133 is a cell surface glycoprotein that has been used as a marker of hematopoietic stem cells, neural stem cells, and for enrichment of a tumor initiating cell population in many cancers including colon carcinoma and glioblastoma. The cancer stem cell compartment contains more than one stem cell population and CD133KDEL is highly effective against at least one of these populations expressing CD133. A fusion polypeptide comprising: a targeting moiety comprising: the monoclonal antibody of embodiment 2; or the scFv of any one of embodiments 3-5; and a toxin moiety comprising a therapeutically active portion of a cytolytic toxin. EXAMPLES Example 1 Cell Culture Caco-2 cells were grown in Minimum Essential Medium (MEM) containing 20% FBS, 1%</s>",Disclosed herein are a monoclonal antibody that specifically binds to human CD133 and single-chain variable fragments thereof. Also disclosed herein is a hybridoma that produces the monoclonal antibody that specifically binds to human CD133.
3,"<pad>The receptors of several hematopoietic growth factors are tyrosine kinases; these include c-fms, which is the colony stimulating factor 1 receptor [Sherr et al., Cell, 41: 665-676 (1985)] and c-kit, a primitive hematopoietic growth factor receptor [Huang et al., Cell, 63: 225-33 (1990)]. The invention is further directed to a method of screening for a neoplastic disease state, comprising the steps of: (

*  Extractive sum actually did well on example 2 where we are describing a synthesis pathway, but the compound names seem to be stored in a list somewhere (compound 2, compound 3, ...) so we cannot directly discern the synthesis from the summary provided.
*  Further, we see references to things like "Formula 1" which indicates a formula listed somewhere in the document that isn't easily accessed by the model.
